<a href="https://colab.research.google.com/github/sam-thurman/asl_alphabet_image_classification/blob/master/notebooks/model_create/edge_model_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd drive/My\ Drive/

/content/drive/My Drive


In [0]:
import os
import pathlib
import tarfile

import boto3
from botocore.handlers import disable_signing

from scipy.io import loadmat
from imageio import imread

import numpy as np
from skimage.transform import resize


def list_files(base_path, validExts=(".jpg", ".jpeg", ".png", ".bmp"), contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(base_path):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename).replace(" ", "\\ ")
                yield imagePath


class BSDS500(object):
    BUCKET = 'i008data'
    FN = 'BSR_bsds500.tgz'
    STORE_FN = os.path.join(os.environ['HOME'], 'BSR.tgz')
    HOME = os.environ['HOME']

    def __init__(self, path_to_bsds=None, images_to_gray=False, target_size=None, masks_to_binary=True):
        if not path_to_bsds:
            self.BSDS_BASE = self.get_bsds()
        else:
            self.BSDS_BASE = path_to_bsds
            
        print(self.BSDS_BASE)

        self.images_to_gray = images_to_gray
        self.target_size = target_size
        self.masks_to_binary = masks_to_binary

        self.TRAIN_PATH = os.path.join(self.BSDS_BASE, 'BSDS500/data/images/train/')
        self.TEST_PATH = os.path.join(self.BSDS_BASE, 'BSDS500/data/images/test/')
        self.VALID_PATH = os.path.join(self.BSDS_BASE, 'BSDS500/data/images/val/')
        self.GROUND_TRUTH_TRAIN = os.path.join(self.BSDS_BASE, 'BSDS500/data/groundTruth/train/')
        self.GROUND_TRUTH_TEST = os.path.join(self.BSDS_BASE, 'BSDS500/data/groundTruth/test/')
        self.GROUND_TRUTH_VALID = os.path.join(self.BSDS_BASE, 'BSDS500/data/groundTruth/val/')

    def get_bsds(self):
        if not pathlib.Path(self.STORE_FN).exists():
            print("DOWNLOADING BSDS500 DATA BE PATIENT")
            s3_resource = boto3.resource('s3')
            s3_resource.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)
            bucket = s3_resource.Bucket(self.BUCKET)
            bucket.download_file(self.FN, self.STORE_FN)
            
        ds_dir = self.STORE_FN.split('.')[0]
        
        if not pathlib.Path(ds_dir).is_dir():
            tar = tarfile.open(self.STORE_FN)
            tar.extractall(self.HOME)

        # dir_path = os.path.dirname(os.path.realpath(__file__))
        dir_path = 'bsds500/data'
        return os.path.join(dir_path, self.STORE_FN.split('.')[0])

    def load_ground_truth(self, gt_path):
        ground_truth_paths = sorted(list(list_files(gt_path, validExts=('.mat'))))
        file_id = []
        cnts = []
        sgmnts = []

        for gt_path in ground_truth_paths:
            file_name = os.path.basename(gt_path).split('.')[0]
            gt = loadmat(gt_path)
            gt = gt['groundTruth'][0]
            for annotator in gt:
                contours = annotator[0][0][1]  # 1-> contours
                segments = annotator[0][0][0]  # 0 -> segments
                if self.target_size:
                    contours = resize(contours.astype(float), output_shape=self.target_size)
                    segments = resize(segments, output_shape=self.target_size)

                if self.masks_to_binary:
                    contours[contours > 0] = 1

                file_id.append(file_name)
                cnts.append(contours)
                sgmnts.append(segments)

        cnts = np.concatenate([np.expand_dims(a, 0) for a in cnts])
        sgmnts = np.concatenate([np.expand_dims(a, 0) for a in sgmnts])
        cnts = cnts[..., np.newaxis]
        sgmnts = sgmnts[..., np.newaxis]

        return file_id, cnts, sgmnts

    def load_images(self, list_of_files):
        processed_images = []
        for i, f in enumerate(list_of_files):

            if self.images_to_gray:
                im = imread(f, mode='L')
            else:
                im = imread(f)

            if self.target_size:
                im = resize(im, output_shape=self.target_size)

            processed_images.append(np.expand_dims(im, 0))

        processed_images = np.concatenate(processed_images)

        if self.images_to_gray:
            processed_images = processed_images[..., np.newaxis]

        return processed_images

    def get_train(self):
        file_ids, cnts, sgmnts = self.load_ground_truth(self.GROUND_TRUTH_TRAIN)
        image_paths = [self.TRAIN_PATH + f_id + '.jpg' for f_id in file_ids]
        images = self.load_images(image_paths)

        return file_ids, cnts, sgmnts, images

    def get_test(self):
        file_ids, cnts, sgmnts = self.load_ground_truth(self.GROUND_TRUTH_TEST)
        image_paths = [self.TEST_PATH + f_id + '.jpg' for f_id in file_ids]
        images = self.load_images(image_paths)

        return file_ids, cnts, sgmnts, images

    def get_val(self):
        file_ids, cnts, sgmnts = self.load_ground_truth(self.GROUND_TRUTH_VALID)
        image_paths = [self.VALID_PATH + f_id + '.jpg' for f_id in file_ids]
        images = self.load_images(image_paths)

        return file_ids, cnts, sgmnts, images



In [0]:
import pandas as pd
import numpy as np
import scipy as math
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
# from PIL import Image

from sklearn.model_selection import train_test_split
import keras

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from keras.models import Model, Sequential
from keras import layers
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D, concatenate, UpSampling2D
from keras import optimizers
from keras.optimizers import Adam

import sys
import os

# from bsds500 import BSDS500
from keras.callbacks import CSVLogger

In [0]:
TARGET_SHAPE = (192, 192)
 
bsds = BSDS500(target_size=TARGET_SHAPE)
ids, contours_train, sgmnts, train_images = bsds.get_train()
ids, contours_test, sgmnts, test_images = bsds.get_train()
 
C = np.concatenate([contours_test, contours_train])
I = np.concatenate([test_images, train_images])

/root/BSR


In [0]:
def get_unet(img_dim, channels):
    inputs = Input((img_dim, img_dim, channels))
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
 
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
 
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
 
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
 
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(conv5)
 
    up6 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(conv6)
 
    up7 = UpSampling2D(size=(2, 2))(conv6)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv7)
 
    up8 = UpSampling2D(size=(2, 2))(conv7)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv8)
 
    up9 = UpSampling2D(size=(2, 2))(conv8)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv9)
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
 
    model = Model(inputs=[inputs], outputs=[conv10])
 
    model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy')
    return model

In [0]:
I[0].shape, C[0].shape

((192, 192, 3), (192, 192, 1))

In [0]:
csv_callback = CSVLogger('history.log', append=True)
unet = get_unet(192, 3)
history = unet.fit(I, C, verbose=2, epochs=200, validation_split=0.1, callbacks=[csv_callback])

Train on 1956 samples, validate on 218 samples
Epoch 1/200
 - 20s - loss: 0.4371 - val_loss: 0.3904
Epoch 2/200
 - 11s - loss: 0.3710 - val_loss: 0.3472
Epoch 3/200
 - 11s - loss: 0.3504 - val_loss: 0.3541
Epoch 4/200
 - 11s - loss: 0.3437 - val_loss: 0.3340
Epoch 5/200
 - 11s - loss: 0.3340 - val_loss: 0.3247
Epoch 6/200
 - 11s - loss: 0.3237 - val_loss: 0.3139
Epoch 7/200
 - 11s - loss: 0.3147 - val_loss: 0.3068
Epoch 8/200
 - 11s - loss: 0.3028 - val_loss: 0.2979
Epoch 9/200
 - 11s - loss: 0.2921 - val_loss: 0.2910
Epoch 10/200
 - 11s - loss: 0.2796 - val_loss: 0.2794
Epoch 11/200
 - 11s - loss: 0.2675 - val_loss: 0.3117
Epoch 12/200
 - 11s - loss: 0.2664 - val_loss: 0.2615
Epoch 13/200
 - 11s - loss: 0.2485 - val_loss: 0.2548
Epoch 14/200
 - 11s - loss: 0.2397 - val_loss: 0.2494
Epoch 15/200
 - 11s - loss: 0.2340 - val_loss: 0.2369
Epoch 16/200
 - 11s - loss: 0.2238 - val_loss: 0.2335
Epoch 17/200
 - 11s - loss: 0.2192 - val_loss: 0.2241
Epoch 18/200
 - 11s - loss: 0.2154 - val_los

In [0]:
unet.save('unet.h5')

In [0]:
%ls

 Accounts.gdoc
 asl_alphabet_image_classification/
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
 bsds500/
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 copy_base__inceptionv3.h5
 copy_base_model.h5
 copy_model1.h5
 copy_model2.h5
 copy_model3.h5
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'fly by night.mp3'
'Getting started.pdf'
 history.log
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
 model6.h5
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.m

In [0]:
del unet

In [0]:
from keras.models import load_model
unet = load_model('unet.h5')

In [0]:
del unet


In [0]:
%ls

 Accounts.gdoc
 asl_alphabet_image_classification/
'bb logo v2 (05:29:19).pdf'
'Behavioral quizz write-up 301.gdoc'
 bsds500/
'Capstone MVP presentation.gslides'
'ceative resume.gdoc'
'CF 301 reading journal.gdoc'
 Codefellows/
'(codefellows)Answers to Common Behavioral Questions.gdoc'
 communication.mp3
 copy_base__inceptionv3.h5
 copy_base_model.h5
 copy_model1.h5
 copy_model2.h5
 copy_model3.h5
 data/
 Fin:BobbyBrooka_AsIShould.mp3
 Fin_Mischief.mp3
'Fin_Wreckage 3.mp3'
'Flatiron Capstone Proposal.gdoc'
'flatiron notes'/
'fly by night.mp3'
'Getting started.pdf'
 history.log
"hot n' cold beat RW.mp3"
'ice man (concept not finished).mp3'
'Image 11-18-19 at 11.12 AM.JPG'
'in my ceiling.mp3'
'in my ceiling (w VERY ROUGH voc concept).mp3'
'Kaia_RedWine_BetweenUs 2.mp3'
 Kaia_RedWine_BetweenUs.mp3
'mischief (1).mp3'
 mischief.mp3
 model6.h5
'Norwood_TollFree[Master].mp3'
'nothing nowhere.mp3'
'pain on your thigh'/
'pain on your thigh 1-Drum Rack.mp3'
'pain on your thigh 2-Addictive Keys.m